# Knapsack Problem
by Božidar Benko, CTO @ VuMedi.com

* imagine you are a thief and you break in a bank vault
* there are some items in the vault
* each item has a mass and a value on (black) market
* you have only one knapsack with limited capacity
* if you take more stuff than your knapsack capacity, it will break, you'll loose all items and get shot

What is the maximum profit you can get? 

![Knapsack Problem](https://upload.wikimedia.org/wikipedia/commons/thumb/f/fd/Knapsack.svg/486px-Knapsack.svg.png)
From CC BY-SA 2.5, https://commons.wikimedia.org/w/index.php?curid=985491

## Definition
Given a set of items, each with a mass and a value, determine the number of each item to include in a collection so that the total weight is less than or equal to a given limit and the total value is as large as possible.

### Main Variations
* 0-1 knapsack problem
   * only one item of a kind
   * take an item or leave it (no fractions)
* bounded knapsack problem (BKP)
   * take up to *c* items of each kind
* unbounded knapsack problem (UKP)
   * no upper bound on items
   
## 0-1 Knapsack
Given a set of *n* items numbered from 0 up to *n-1*, each with a mass *m[i]* and a value *v[i]*, along with a maximum mass capacity *M*,

maximize

```python
V = sum([v[i]*x[i] for i in range(n)])  # total value in knapsack
```

with
```python 
sum([m[i]*x[i] for i in range(n)]) <= M  # items fit in backpack
and
all([k in [0, 1] for k in x])  # take (1) or leave (0) item, cannot break it
```

### Example


In [1]:
M = 7
n = 4
m = [2, 3, 4, 5]
v = [3, 4, 5, 5]
# V = ?

### Solution:

```python
V = 9  # v[1] + v[2]

m[1] + m[2] <= M  # 7

x = [0, 1, 1, 0]  # pick second and third item 
```

#### Simple Greedy Solution?
* let's calculate value per kilogram *v[i]/m[i]*
* sort in descending order
* and pick items from top

In [2]:
[v[i]/m[i] for i in range(n)]  # it's already sorted in this example

[1.5, 1.3333333333333333, 1.25, 1.0]

If we pick first item (best value per kilogram), then the best we can do is:

V = 8

Doesn't work!

* there is no known algorithm both correct and fast (polynomial-time) on all cases (NP-complete)
* there is a pseudo-polynomial time algorithm using dynamic programming

#### Recursive Solution
* try all (2**n) subsets of items
* calculate total mass for each subset
* consider only the subsets where total mass is <= M
* pick the optimal subset with maximum value

#### Optimal Substructure
In optimal substructure (subset), every item can be either:
* not included
* included

So, maximum value of n items is the better of:
* nth item not included
    * maximum of n - 1 items
    * M kg available capacity in backpack
* nth item included
    * value of nth item + maximum of n - 1 items
    * M - mass of nth item capacity in backpack

If nth item cannot be included (mass greater than capacity), then we have just the first case.



In [3]:
def knapsack_recursive(M, n, m, v):
    # print(" "*M, M, n)
    if n == 0 or M == 0:  # base case
        return 0
    if m[n-1] > M:  # last item doesn't fit
        return knapsack_recursive(M, n-1, m, v)  # not included
    else:
        return max(
            knapsack_recursive(M, n-1, m, v),  # not included
            v[n-1] + knapsack_recursive(M - m[n-1], n-1, m, v)  # included
        )

knapsack_recursive(M, n, m, v)


9

Time complexity: O(2**n)

Space complexity: O(1)

Optimization: overlapping subproblems can be memoized (store the solution for knapsack_recursive(M', n', m, v) in a cache table - reuse it after, instead of recalculating


#### Dynamic Programming Solution
* solve (smaller) subproblems first (just once)
* store the results of subproblems in memory table
* use their solutions to solve bigger and bigger subproblems
* bottom-up approach

Definition:

d[i][j] - maximum value that can be attained by
    * using first i items (0 <= i <= n)
    * with mass less or equal than j  (0 <= j <= M)

Recursive formula for d[i][j]

```python
* d[0][j] = 0  # using zero items
* if m[i-1] > j  # item is heavier than current mass limit
    d[i][j] = d[i-1][j]
* else if m[i-1] <= j
    d[i][j] = max(
                d[i-1][j],  # don't take
                v[i-1] + d[i-1][j-m[i-1]]  # take - gain value + the best of remaining capacity
              )
```

Then the solution is:

V = d[n][M]


In [4]:
def knapsack(M, n, m, v, debug=False):
    d = [[0 for j in range(M+1)] for i in range(n+1)]
    
    for i in range(1, n+1):  # start considering 1, then 2, ... items; actual index of item is i-1
        for j in range(M+1):
            if m[i-1] > j:  # if item is heavier than current capacity
                d[i][j] = d[i-1][j]  # take the best value of previous choices 
            else:
                d[i][j] = max(  # the best value of:
                    d[i-1][j],  # don't include item - take previous best
                    v[i-1] + d[i-1][j-m[i-1]]  # value of item + best value of previous items with lowered capacity 
                )
        if debug:
            from util import print_knapsack_table
            print_knapsack_table(d, i, m, v)
    return d[n][M]
knapsack(M, n, m, v, True)


,,,0 <= j <= M kg
i,mass,value,"[0, 1, 2, 3, 4, 5, 6, 7]"
0,-,-,"[0, 0, 0, 0, 0, 0, 0, 0]"
1,2kg,$3,"[0, 0, 3, 3, 3, 3, 3, 3]"


,,,0 <= j <= M kg
i,mass,value,"[0, 1, 2, 3, 4, 5, 6, 7]"
0,-,-,"[0, 0, 0, 0, 0, 0, 0, 0]"
1,2kg,$3,"[0, 0, 3, 3, 3, 3, 3, 3]"
2,3kg,$4,"[0, 0, 3, 4, 4, 7, 7, 7]"


,,,0 <= j <= M kg
i,mass,value,"[0, 1, 2, 3, 4, 5, 6, 7]"
0,-,-,"[0, 0, 0, 0, 0, 0, 0, 0]"
1,2kg,$3,"[0, 0, 3, 3, 3, 3, 3, 3]"
2,3kg,$4,"[0, 0, 3, 4, 4, 7, 7, 7]"
3,4kg,$5,"[0, 0, 3, 4, 5, 7, 8, 9]"


,,,0 <= j <= M kg
i,mass,value,"[0, 1, 2, 3, 4, 5, 6, 7]"
0,-,-,"[0, 0, 0, 0, 0, 0, 0, 0]"
1,2kg,$3,"[0, 0, 3, 3, 3, 3, 3, 3]"
2,3kg,$4,"[0, 0, 3, 4, 4, 7, 7, 7]"
3,4kg,$5,"[0, 0, 3, 4, 5, 7, 8, 9]"
4,5kg,$5,"[0, 0, 3, 4, 5, 7, 8, 9]"


9

Time complexity: O(M*n), pseudo-polynomial time

Space complexity: O(M*n)

Optimization: we can use 1-dimensional array to store current optimal values - we get O(M) space complexity

In [5]:
from random import randint
M2 = 100
n2 = 23
m2 = [randint(1, 10) for i in range(n2)]
v2 = [randint(1, 10) for i in range(n2)]
print("Recursive")
%time print(knapsack_recursive(M2, n2, m2, v2))
print()
print("Dynamic Programming")
%time print(knapsack(M2, n2, m2, v2, False))

Recursive
108
CPU times: user 10.8 s, sys: 87 ms, total: 10.9 s
Wall time: 11.2 s

Dynamic Programming
108
CPU times: user 2.1 ms, sys: 27 µs, total: 2.12 ms
Wall time: 2.14 ms


## Bonus: Subset Sum Problem
Given a set of (positive in this case) numbers and a target sum S, is there a non-empty subset with sum of items equal to S.

Can be solved a special case of 0-1 Knapsack with m = v.

In [6]:
M3 = 20
n3 = 5
m3 = v3 = [randint(1, 10) for i in range(n3)]
print(m3)
V = knapsack(M3, n3, m3, v3)
print(V, V==M3)

[10, 4, 9, 4, 3]
20 True


## Questions?
P.S. VuMedi is hiring!